In [ ]:
"""
Purpose: To check on the status of the Allen split suggestions pipeline
as they run through

- Downloading
- Decimation
- Soma Finding
- Decomposition
- Suggestion Splits

"""

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty/")

In [4]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

import system_utils as su

INFO - 2021-03-05 18:06:55,641 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-05 18:06:55,643 - settings - Setting database.user to celiib
INFO - 2021-03-05 18:06:55,644 - settings - Setting database.password to newceliipass
INFO - 2021-03-05 18:06:55,648 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-03-05 18:06:55,649 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-05 18:06:55,663 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-03-05 18:06:56,060 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-05 18:06:56,064 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-05 18:06:56,065 - settings - Setting database.user to celiib
INFO - 2021-03-05 18:06:56,066 - settings - Setting database.password to newceliipass
INFO - 2021-03-05 18:06:56,067 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-05 18:06:56,070 - settings - Setting database.user to celiib
INFO - 2021-03-05 18:06:56,071 - settings - Setting database.password to newceliipass
INFO - 2021-03-05 18:06:56,072 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-05 18:06:56,492 - settings - Setting enable_python_native_blobs to True


In [5]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-03-05 18:06:56,649 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-05 18:06:56,651 - settings - Setting database.user to celiib
INFO - 2021-03-05 18:06:56,652 - settings - Setting database.password to newceliipass
INFO - 2021-03-05 18:06:56,656 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-05 18:06:57,113 - settings - Setting enable_python_native_blobs to True


# All the cells that need to get suggestions

In [6]:
key_source = minnie.Decomposition() & "n_somas>1 OR n_error_limbs>0" & (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj()
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134988608634,0,0.25,30.00,5,0,1,=BLOB=,1150159,2311814,0,1,1,1,2,11,346,233,11067259.173807988,5580597.123682757,15796.577,104.973,104.973,450.146,379.101,4184,0.00037805204832484123,12.092485549132949,6872596.828,156,0.0006087946237372387,26.821,946998934578.9817,136631721.2458318,85567.61161066596,137793.46559669625,6070505990.890908,4574.8436
864691135065024068,0,0.25,30.00,5,0,1,=BLOB=,1683573,3378174,0,2,2,0,1,11,346,101,15448645.134033982,4735340.661534153,28195.927,309.528,279.925,621.111,612.357,4401,0.00028487935102505666,12.719653179190752,12951735.977,254,0.00033980000888030764,17.327,644008505500.065,97079430.94166055,41687.054101675785,49723.72094703834,2535466557.0868697,7567.1259
864691135155711844,0,0.25,30.00,6,0,1,=BLOB=,1323348,2681606,0,2,2,0,1,12,247,99,10335501.970505765,4157038.1039918233,26480.998,386.808,304.693,553.981,502.772,6481,0.0006270619480790302,26.238866396761132,8914214.834,199,0.0007270410373419097,32.568,1301368737143.5571,126957093.46514548,125912.48503045616,145988.0383609293,6539541392.681191,3068.9409
864691135210232512,0,0.25,30.00,6,0,1,=BLOB=,1104606,2279207,0,5,4,2,2,15,374,97,11682255.066969572,3937393.8053746116,18011.375,128.134,128.134,454.419,431.091,2282,0.00019533899807171054,6.101604278074866,7371490.013,204,0.0003095710631060445,11.186,390388762128.02075,108385813.17371431,33417.24349366473,52959.27437187735,1913670402.588337,2271.6559
864691135341057989,0,0.25,30.00,6,0,1,=BLOB=,910711,1829521,0,3,1,2,2,14,259,112,10163119.200359212,4474894.858952958,28470.197,143.696,143.696,469.631,412.785,3107,0.0003057132302344918,11.996138996138995,6203039.418,137,0.0005008834847936154,22.679,557754542849.0073,106585138.54347737,54880.25200268178,89916.3305702223,4071201042.6934843,2310.7131
864691135373676873,0,0.25,30.00,6,0,1,=BLOB=,690741,1403169,0,2,0,2,2,10,203,108,9240166.369001659,3725968.734

In [8]:
no_splits = (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)) - key_source.proj()
no_splits

segment_id segment id for those to be decimated,month the month that the cell was added,day the day that the cell was added,year the year that the cell was added,n_somas the number of somas associated with the neuron from the Allen Institute
864691135345291423,3,3,2021,2
864691135373423816,3,3,2021,2
864691135564804951,3,3,2021,2
864691135753669837,3,3,2021,2
864691135782479155,3,3,2021,2
864691135939404545,3,3,2021,2
864691135975576131,3,3,2021,2
864691136008578990,3,3,2021,2
864691136008729518,3,3,2021,2
864691136922748900,3,3,2021,2


In [ ]:
minnie.DecompositionSplit() & key_source.proj()

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition_split'")
(curr_table)#.delete()# & "status='error'")

# Getting the split suggesstions

In [12]:
minnie.DecompositionSplit() & key_source.proj()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,split_version the version of the splitting algorithm used,multiplicity the number of somas found for this base segment,n_splits the number of cuts required to help split the neuron,split_success the successfulness of the splitting,n_error_limbs_cancelled number of limbs that couldn't be resolved and cancelled out,n_same_soma_limbs_cancelled number of same soma touching limbs that couldn't be resolved and cancelled out,n_multi_soma_limbs_cancelled number of multi soma touching limbs that couldn't be resolved and cancelled out,error_imbs_cancelled_area the total area (in microns^2) of the limbs that was cancelled out because touching the same soma multiple times or multiple somas,error_imbs_cancelled_skeletal_length the total skeletal length (in microns) of the limbs that were called out because could not be resolved,split_results will store the results of how to split the limbs of neuron objects from original neuron,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,max_soma_n_faces The largest number of faces of the somas,max_soma_volume The largest volume of the somas the (volume in billions (10*9 nm^3)),n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134988608634,0,0.25,30.00,5,0,0,1,2,5,0,0,0,0,0.0,0.0,=BLOB=,=BLOB=,574080,1144852,0,0,0,0,1,42748,873,7,199,121,6223315.026890338,2352247.931240865,15303.421,83.572,83.572,430.775,372.965,1824,0.0002930913817023039,9.165829145728644,3332797.258,70,0.0005472880162817273,26.057,521748426666.5095,184150767.40581003,83837.70135564171,156549.70473049686,7453548952.378708,2564.3971
864691134988608634,0,0.25,30.00,5,0,1,1,2,5,0,0,0,0,0.0,0.0,=BLOB=,=BLOB=,520895,1040511,0,0,0,0,1,36394,693,9,129,46,4443937.522417968,1396097.3291300046,17858.904,196.352,191.43,460.111,399.887,2337,0.0005258849811030706,18.11627906976744,3376842.276,79,0.000692066673237776,29.582,420208746291.92224,116005394.72003174,94557.752932423,124438.36932463298,5319098054.32813,3980.8336
8646911350650

In [26]:
allen_segment_ids = (minnie.AllenProofreading() & dict(day=3,month=3,year=2021)).fetch("segment_id")
du.create_suggested_splits_neuroglancer_spreadsheet(segment_ids=allen_segment_ids,output_type="server",
                                                   print_links=True)

  0%|          | 0/34 [00:00<?, ?it/s]

segment_id = 864691134988608634
segment_id = 864691135065024068
segment_id = 864691135155711844
segment_id = 864691135210232512
segment_id = 864691135341057989
segment_id = 864691135382990571
segment_id = 864691135463533125
segment_id = 864691135472075826
segment_id = 864691135510515408
segment_id = 864691135571307557
segment_id = 864691135609490439
segment_id = 864691135653543364
segment_id = 864691135753592525
segment_id = 864691135758491982
segment_id = 864691135775809453
segment_id = 864691135807230237
segment_id = 864691135841095907
segment_id = 864691135850464967
segment_id = 864691135850470343
segment_id = 864691135883914352
segment_id = 864691135945417764
segment_id = 864691136008436140
segment_id = 864691136008496556
segment_id = 864691136008511148
segment_id = 864691136031686843
segment_id = 864691136041004886
segment_id = 864691136056568536
segment_id = 864691136090951092
segment_id = 864691136236730639
segment_id = 864691136311833405
segment_id = 864691136486808850
segment_